# 不均衡データの評価

## 不均衡データとは<a name="description"></a>

$A:B=1:99$ などクラス間のサンプル数に極端な偏りのあるデータ。$A$を無視して全て$B$と予測するモデルでも精度99%を達成してしまう。精度をモデルの評価指標とできないので、他の指標が必要。

不均衡データを評価するには、まず混同行列を理解しないといけない。

### 混同行列(confusion matrix)<a name="confusion_matrix"></a>

In [ ]:
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

loader = load_breast_cancer()
X, y = loader.data, loader.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=0, stratify=y)
y_pred = LogisticRegression(random_state=0, n_jobs=-1).fit(X_train, y_train).predict(X_test)
labels, labels_str = [1, 0], ['1 (True)', '0 (False)']
confmat = confusion_matrix(y_true=y_test, y_pred=y_pred, labels=labels)

fig, axes = plt.subplots(1, 2, figsize=(8, 4))

ax1, ax2 = axes[0], axes[1]

ax1.matshow(np.zeros((2, 2)), alpha=0)
for row in range(2):
    for col in range(2):
        value = 'True ' if row == col else 'False '
        value += 'Positive' if col == 0 else 'Negative'
        ax1.text(x=col, y=row, s=value, va='center', ha='center')

ax2.matshow(confmat, alpha=.3)
for row, cols in enumerate(confmat):
    for col, value in enumerate(cols):
        ax2.text(x=col, y=row, s=value, va='center', ha='center')

for i, ax in enumerate(axes):
    if i == 0:
        ax.set_ylabel('true label')
        ax.set_yticks([0, 1])
        ax.set_yticks([.5], minor=True)
        ax.set_yticklabels(labels_str)
    else:
        ax.set_yticklabels(())
    ax.set_title('Confusion Matrix')
    ax.set_xlabel('predicted')
    ax.set_xticks([0, 1])
    ax.set_xticks([.5], minor=True)
    ax.set_xticklabels(labels_str)
    ax.grid(linestyle='-', which='minor')

plt.show()

## 適合率(precision)・再現率(recall)・F1スコア<a name="f1"></a>

適合率(precision)とは、Trueと分類したものの中での正解率

$\begin{eqnarray}
    PRE=\frac{TP}{TP+FP} \nonumber
\end{eqnarray}$

再現率(recall)とは、正解ラベルがTrueのものの中での正解率

$\begin{eqnarray}
    REC=\frac{TP}{TP+FN} \nonumber
\end{eqnarray}$

F1スコアとは、適合率と再現率を組み合わせた指標。適合率・再現率ともに高くないと値が大きくならない。

$\begin{eqnarray}
    F1=2\times\frac{PRE\times REC}{PRE+REC} \nonumber
\end{eqnarray}$

In [ ]:
division = 10
values = np.linspace(.1, 1, division)

M = np.zeros((division, division, 2))
for i, v in enumerate(values):
    M[:, i, 0] = v
    M[i, :, 1] = v
PRE, REC = M[:, :, 0], M[:, :, 1]
F1 = 2 * (PRE * REC) / (PRE + REC)

fig = plt.figure(figsize=(4, 4))
ax = fig.add_subplot(1, 1, 1)

ax.set_title('F1 score')

ax.matshow(F1, cmap='Reds', alpha=.6)
for row, cols in enumerate(F1):
    for col, value in enumerate(cols):
        ax.text(x=col, y=row, s='%.2f' % value, va='center', ha='center', size='x-small')

ticks = np.arange(division)
ticklabels = ['%.1f' % v for v in values]
ax.set_xticks(ticks)
ax.set_xticklabels(ticklabels, size='x-small')
ax.set_xlabel('Recall')
ax.set_yticks(ticks)
ax.set_yticklabels(ticklabels, size='x-small')
ax.set_ylabel('Precision')

plt.show()

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

print('Precision: %.3f' % precision_score(y_true=y_test, y_pred=y_pred))
print('Recall: %.3f' % recall_score(y_true=y_test, y_pred=y_pred))
print('F1: %.3f' % f1_score(y_true=y_test, y_pred=y_pred))

## 受信者操作特性(Reciever Operator Characteristic, ROC)<a name="roc"></a>

真陽性率(True Positive Rate)と偽陽性率(False Positve Rate)に基づく指標。

真陽性率とは、正解ラベルがTrueのものの中での正解率。

$\begin{eqnarray}
TPR=\frac{TP}{TP+FN} \nonumber
\end{eqnarray}$

偽陽性率とは、正解ラベルがFalseのものの中での不正解率。

$\begin{eqnarray}
    FPR\ =\ \frac{FP}{FP+TN} \nonumber
\end{eqnarray}$

通常、分類器の出力が0.5以上ならTrue・0.5未満ならFalseと予測するが、この閾値を変化させた場合のTPR・FPRの変化を描画したものがROC曲線

In [ ]:
x = np.linspace(0, 1, 50)
roc = np.sqrt(1 - (x - 1) ** 2)

plt.figure(figsize=(4, 4))
plt.title('Receiver Operator Characteristic Curve')

plt.fill_between(x, 0, roc, alpha=.1)
plt.plot(x, roc, label='ROC')
plt.plot([0, 0, 1], [0, 1, 1], linestyle='--', label='Perfect performance')
plt.plot([0, 1], [0, 1], linestyle='--', label='Random guessing')

plt.legend(loc='lower right', fontsize='small')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.xlim(-.05, 1.05)
plt.ylim(-.05, 1.05)
plt.grid()

plt.show()

グラフの四隅は

- 左上…TPR=1, FPR=0→全てのサンプルで分類成功
- 左下…TPR=0, FPR=0→全てのサンプルをFalseと予測
- 右上…TPR=1, FPR=1→全てのサンプルをTrueと予測
- 右下…TPR=0, FPR=1→全てのサンプルで分類失敗

を表す。

ROC曲線の下部分の面積(Area Under the Curve, AUC)を、その分類器の性能と捉えられる。

In [ ]:
from sklearn.metrics import roc_curve, auc

model = LogisticRegression(random_state=0, n_jobs=-1).fit(X_train[:, -2:], y_train)
probas = model.predict_proba(X_test[:, -2:])
fpr, tpr, thresholds = roc_curve(y_test, probas[:, 1], pos_label=1)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(3, 3))

plt.plot(fpr, tpr)

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')

plt.show()

In [ ]:
from sklearn.metrics import roc_auc_score, accuracy_score

pred = model.predict(X_test[:, -2:])
print('ROC AUC: {roc:.3f}, Accuracy: {acc:.3f}'.format(
    roc=roc_auc_score(y_true=y_test, y_score=probas[:, 1]),
    acc=accuracy_score(y_true=y_test, y_pred=pred)))

## マクロ平均法とマイクロ平均法<a name="multi_class"></a>

多クラス分類のための指標。

一対全で$k$個のクラスそれぞれの混同行列を作成したとして

マクロ平均法は、各クラスで指標を求めてから、それらの平均を最終的な指標として使用する。各クラスを平等に扱う。

$\begin{eqnarray}
    PRE_{macro}=\frac{1}{k}\left(PRE_1+...+PRE_k\right) \nonumber
\end{eqnarray}$

マイクロ平均法は、それぞれの分母・分子同士を合計してから、最終的な指標の分母・分子として使用する。各サンプルを平等に扱う。

$\begin{eqnarray}
    PRE_{micro}=\frac{TP_1+...+TP_k}{TP_1+...+TP_k+FP_1+...FP_k} \nonumber
\end{eqnarray}$

scikit-learnで二値分類の指標を使って多クラス分類モデルを評価すると、デフォルトではマクロ平均を正規化(重みづけ)したものが使用される。それ以外の平均化方法を使用したい場合はaverage引数で指定。